In [2]:
import spreg
import libpysal
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from pyarrow import parquet

In [3]:
import pandas as pd


In [4]:
shape = gpd.read_parquet("shape_final.parquet")

In [5]:
shape['log_populacao'] = np.log(shape['populacao'])

In [60]:
# OLS

y = 'ice'
x = [
    'PLP',
    'PLB',
    'diversidade_credito',
    'diversidade_bancaria',
    'log_populacao',
    'op_cred'
]

In [61]:
shape = shape.dropna()

In [62]:
Y = shape[y]
X = shape[x]

In [63]:
shape["centroid"] = shape.geometry.centroid

# 3. Separar coordenadas dos centróides
shape["centroid_x"] = shape["centroid"].x
shape["centroid_y"] = shape["centroid"].y

<positron-console-cell-63>:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [71]:
coords = shape[["centroid_x", "centroid_y"]].values
knn = libpysal.weights.KNN.from_array(coords, k=5)

In [72]:
w_sparse = knn.sparse
knn.transform = "R"

In [68]:
ols1 = spreg.OLS(Y, X)


In [69]:
print(ols1.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :         ice                Number of Observations:        2636
Mean dependent var  :      0.6071                Number of Variables   :           7
S.D. dependent var  :      0.8719                Degrees of Freedom    :        2629
R-squared           :      0.6080
Adjusted R-squared  :      0.6071
Sum squared residual:     785.174                F-statistic           :    679.7170
Sigma-square        :       0.299                Prob(F-statistic)     :           0
S.E. of regression  :       0.546                Log likelihood        :   -2144.076
Sigma-square ML     :       0.298                Akaike info criterion :    4302.153
S.E of regression ML:      0.5458                Schwarz criterion     :    4343.292

------------------------------------------------------------

In [96]:
ols1b = spreg.OLS(Y, X, w=knn, spat_diag=True, moran=True)
print(ols1b.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :         ice                Number of Observations:        2636
Mean dependent var  :      0.6071                Number of Variables   :           7
S.D. dependent var  :      0.8719                Degrees of Freedom    :        2629
R-squared           :      0.6080
Adjusted R-squared  :      0.6071
Sum squared residual:     785.174                F-statistic           :    679.7170
Sigma-square        :       0.299                Prob(F-statistic)     :           0
S.E. of regression  :       0.546                Log likelihood        :   -2144.076
Sigma-square ML     :       0.298                Akaike info criterion :    4302.153
S.E of regression ML:      0.5458                Schwarz criterion     :    4343.292

------------------------------------------------------------

In [97]:
ols2c = spreg.OLS(Y, X, w=knn, spat_diag=True, moran = True, slx_lags=1)

print(ols2c.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES WITH SPATIALLY LAGGED X (SLX)
-----------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :         ice                Number of Observations:        2636
Mean dependent var  :      0.6071                Number of Variables   :          13
S.D. dependent var  :      0.8719                Degrees of Freedom    :        2623
R-squared           :      0.6587
Adjusted R-squared  :      0.6571
Sum squared residual:     683.716                F-statistic           :    421.8360
Sigma-square        :       0.261                Prob(F-statistic)     :           0
S.E. of regression  :       0.511                Log likelihood        :   -1961.714
Sigma-square ML     :       0.259                Akaike info criterion :    3949.429
S.E of regression ML:      0.5093                Schwarz criterion     :    4025.830



In [82]:
ak = spreg.AKtest(ols2c,w=knn)

In [85]:
print(ak.mi)

0.492991549144364


In [89]:
sdm = spreg.GM_Lag(Y, X, w=knn, spat_diag=True, slx_lags=1)

In [90]:
print(sdm.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES WITH SLX (SPATIAL DURBIN MODEL)
----------------------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :         ice                Number of Observations:        2636
Mean dependent var  :      0.6071                Number of Variables   :          14
S.D. dependent var  :      0.8719                Degrees of Freedom    :        2622
Pseudo R-squared    :      0.8098
Spatial Pseudo R-squared:  0.6780

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT        -0.63389         0.23366        -2.71288         0.00667
                 PLP         0.14107         0.04828         2.

In [28]:
ak = spreg.AKtest(ols2c, w=knn)

In [30]:
ak.mi

np.float64(0.492991549144364)

In [120]:
lag3 = spreg.GM_Combo_Het(Y,X,w=knn,name_w="KNN 5",name_ds="MUNICIPIOS",slx_lags=1,w_lags=1)

AttributeError: 'GM_Combo_Het' object has no attribute 'utu'

In [122]:
mod, finreg = spreg.spsearch.stge_classic(Y,X,w=knn)

Model selected by STGE-Classic: SARSAR
REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIALLY WEIGHTED 2SLS- GM-COMBO MODEL (HET)
----------------------------------------------------------------
Data set            :     unknown
Weights matrix      :       False
Dependent Variable  :         ice                Number of Observations:        2636
Mean dependent var  :      0.6071                Number of Variables   :           8
S.D. dependent var  :      0.8719                Degrees of Freedom    :        2628
Pseudo R-squared    :      0.7066
Spatial Pseudo R-squared:  0.6008
N. of iterations    :           1                Step1c computed       :          No

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT        -3.22897         0.09925       -

In [123]:
mod, finreg = spreg.spsearch.stge_pre(Y,X,w=knn,name_ds="shape",name_w="KNN 5")

Model selected by STGE-Pre: SDM
REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES WITH SLX (SPATIAL DURBIN MODEL)
----------------------------------------------------------------------------------
Data set            :       shape
Weights matrix      :       KNN 5
Dependent Variable  :         ice                Number of Observations:        2636
Mean dependent var  :      0.6071                Number of Variables   :          14
S.D. dependent var  :      0.8719                Degrees of Freedom    :        2622
Pseudo R-squared    :      0.8085
Spatial Pseudo R-squared:  0.6699

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT        -0.31076         0.20235        -1.53573         0.12461
                 PLP         0.

In [124]:
mod, finreg = spreg.spsearch.gets_sdm(Y,X,w=knn,robust="white")

Model selected by GETS-SDM: SARSAR
REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIALLY WEIGHTED 2SLS- GM-COMBO MODEL (HET)
----------------------------------------------------------------
Data set            :     unknown
Weights matrix      :       False
Dependent Variable  :         ice                Number of Observations:        2636
Mean dependent var  :      0.6071                Number of Variables   :           8
S.D. dependent var  :      0.8719                Degrees of Freedom    :        2628
Pseudo R-squared    :      0.7066
Spatial Pseudo R-squared:  0.6008
N. of iterations    :           1                Step1c computed       :          No

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT        -3.22897         0.09925       -32.5